In [1]:
!pip install openpyxl
!pip install XlsxWriter
!pip install xlrd
!pip install python-dateutil
!pip install pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 614.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 577.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import glob
import os
import pyreadstat
import gc

In [3]:
# pd.set_option('display.max_rows', None)

# input_root_path = "../data"
input_root_path = "../input/travel-time"
ats_path = "../input/atstraffic"
weather_path = "../input/weatherstations"
counter_path = "../input/counterstations"
# output_root_path = "../data"
output_root_path = "../working"

# Load Dataset


## Add Initial Data

In [4]:
def load_csv_to_df(files, filename=None, is_excel=False):
    if is_excel:
        df = pd.concat((pd.read_excel(f) for f in files), ignore_index=True)
    else:
        df = pd.concat((pd.read_csv(f, skipinitialspace = True) for f in files), ignore_index=True)
    
    if filename:
        df.to_csv(os.path.join(output_root_path, filename), header = True, index=True)
    return df

In [5]:
# Load data 

tasks_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive_All_Data/Update Travel Time Archive/Travel Time Task/TravelTimeTaskArchive*.csv"))
df_tasks_ini = load_csv_to_df(tasks_joined_files)
# df_tasks_ini = load_csv_to_df(tasks_joined_files, filename="all_TravelTimeTaskArchive.csv")

summary_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive_All_Data/Update Travel Time Archive/Travel Time Summary/TravelTimeSummaryTaskArchive*.csv"))
# df_summary = load_csv_to_df(summary_joined_files)

df_tasks_ini

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
1,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
2,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
3,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
4,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
...,...,...,...,...,...,...,...,...
13438804,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438805,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438806,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438807,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06


## Add Nectar Data

In [6]:
def load_nectar_to_df(files, filename=None):
    df_nectar = pd.DataFrame()
    
    for f in files:
        df_temp = pd.read_csv(f, skipinitialspace = True)
        filename = os.path.splitext(f)[0].split('/')
        df_temp.insert(loc=0, column='AssetNumber', value=int(filename[-1]))
        df_temp.insert(loc=1, column='AssetIdentifier', value=filename[-2])
        df_temp.rename(columns = {
            'Task Date': 'CreatedDate', 'Under Construction': 'UnderConstructionTime',
            'Baseline': 'BaselineTime', 'Delay': 'DelayTime', 'Segment Length': 'SegmentLength'
        }, inplace=True)

        df_nectar = pd.concat([df_nectar,
                               df_temp[['AssetNumber', 'AssetIdentifier', 'CreatedDate', 'UnderConstructionTime', 'BaselineTime', 'DelayTime', 'State', 'SegmentLength']]],
                              axis=0, ignore_index=True)
#         df_nectar = pd.concat([df_nectar, df_temp], axis=0)
        
    return df_nectar

In [7]:
nectar_joined_files = glob.glob(os.path.join(input_root_path, "ATSTraffic Dataset/Travel Time/*/*.csv"))
df_nectar = load_nectar_to_df(nectar_joined_files)

df_nectar

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363
...,...,...,...,...,...,...,...,...
1555655,51967047367,Spread 5B,6/4/2022 6:50:36 AM,1.52,1.75,0.00,OK,2.610
1555656,51967047367,Spread 5B,6/4/2022 6:48:40 AM,1.52,1.75,0.00,OK,2.610
1555657,51967047367,Spread 5B,6/4/2022 6:47:18 AM,1.52,1.75,0.00,OK,2.610
1555658,51967047367,Spread 5B,6/4/2022 6:44:35 AM,1.50,1.75,0.00,OK,2.610


## Concat both datasets

In [8]:
df_tasks = pd.concat([df_nectar, df_tasks_ini], axis=0, ignore_index=True)
df_tasks

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363
...,...,...,...,...,...,...,...,...
14994464,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060
14994465,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060
14994466,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060
14994467,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060


## Find Unique Assets


In [9]:
def get_unique_assets(filename=None):
    df_asset = pd.concat([df_tasks[['AssetNumber', 'AssetIdentifier']], df_summary[['AssetNumber', 'AssetIdentifier']]], axis=0) 
    df_asset_unique = df_asset.drop_duplicates()
    
    if filename:
        df_asset_unique.to_csv(os.path.join(output_root_path, filename), header = True, index=True)

    return df_asset_unique

In [10]:
# df_asset_unique = get_unique_assets()
# df_asset_unique = get_unique_assets(filename='unique_assets.csv')
# df_asset_unique

# Get Assets

In [11]:
# df_assets_all_active = pd.read_excel(os.path.join(ats_path, 'AllWorkZones.xlsx'), sheet_name = ['HaveBeenActivated'])['HaveBeenActivated']
# df_assets_all_active

In [12]:
df_assets_recom = pd.read_excel(os.path.join(ats_path, 'Recommended Assets.xlsx'))
df_assets_recom

,Asset Number,Segment Name,Segment Length,Under Construction,Baseline,Direct of Travel,Spread,Traffic Impact,Details of Right of Way to work within or close,Description of work,Road Type,Traffic Control Setup,Easting,Northing,Most Recent Activation,Cell Coverage
0,62134790681,AP 3-B-12-A-1,2.746,1.67,1.72,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
1,10897197432,AP 3-B-12-A-1,2.746,1.68,1.73,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
2,12768050215,AP 3-B-13-A-1,2.970,1.77,1.92,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
3,56678162306,AP 3-B-13-A-1,2.970,1.80,1.92,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
4,86851077697,AP 3-B-13-A-3,18.000,NaN,NaN,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of an Access Point along Spread 3-4A o...,Multi-Lane Undivided Roadways,SLAT,346092.86128,5.857159e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,34684428742,T3920.0,4.840,0.00,0.00,SB,Spread 5A,High,SMJV is requesting approval to work within the...,This application is for traffic control at the...,Multi-Lane Divided,Lane Closure,660846.97920,5.552540e+06,NaN,NaN
620,69027783597,T4068.3,3.460,0.00,0.00,SB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
621,92801985490,T4068.3,3.480,0.00,0.00,NB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
622,16764775025,TWS_460042,3.690,2.73,2.58,SB,Reactivation,High,I/W request approval to work within the Minist...,"The eastbound passing lane will be closed, the...",Multi-Lane Undivided Roadways,Lane Closure,356159.86762,5.876918e+06,4/4/2022,No Cell Coverage Area


In [13]:
# # Remove No Cell Coverage Area entries
# df_assets_recom_no_cell = df_assets_recom[df_assets_recom['Cell Coverage'] != 'No Cell Coverage Area']
# df_assets_recom_no_cell

In [14]:
# df_assets_all_active.columns.difference(df_assets_recom.columns)

# Pre-processing & Cleaning

## Filter Valid Traffic Times

In [15]:
# Filter out entries with zero UnderConstructionTime or BaselineTime
df_valid = df_tasks[(df_tasks['UnderConstructionTime'] != 0) & (df_tasks['BaselineTime'] != 0)].reset_index(drop=True)

# df_valid = df_tasks[(df_tasks['UnderConstructionTime'] != 0) & (df_tasks['BaselineTime'] != 0)].rename_axis('AllTasksIndex').reset_index()
# df_valid = df_valid.rename_axis('ValidTasksIndex')
# df_valid.index.name = 'ValidTasksIndex'
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363
...,...,...,...,...,...,...,...,...
5365467,88582155367,Spread 7 - SB,4/17/2021 6:00:11 PM,21.77,23.60,0.00,OK,32.980
5365468,75946582969,Spread 4B - NB,4/17/2021 6:00:12 PM,43.37,45.35,0.00,OK,70.710
5365469,43132175891,Spread 5B - SB,4/17/2021 6:00:12 PM,53.48,53.73,0.00,OK,87.350
5365470,99369489819,Spread 2 - SB,4/17/2021 6:00:12 PM,165.52,169.87,0.00,OK,286.840


## Match work zone datasets
For all the workzones under the "have been activated" tab, creates a separate spreadsheet for each workzone that contains all the travel time information that you can find in the "Update Travel Time Archive_All_Data" spreadsheet. So for each workzone we would have one spreadsheet with all the travel time information.

In [16]:
def find_workzone_tt_data(df_assets, df_tt, file_name=None):
    df_tt_wz = pd.DataFrame()
    
    for i, location_id in enumerate(df_assets['Asset Number']):
        df_intersect = df_tt[df_tt['AssetNumber'] == location_id]
        df_tt_wz = df_tt_wz.append(df_intersect)
        
        if file_name:
            with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
                df_intersect.to_excel(writer, sheet_name=str(location_id))
            
        if (i+1) % 100 == 0:
            print(f'{round((i+1)/len(df_assets)*100, 2)} %')
            
    return df_tt_wz
         

In [17]:
df_tt_workzone = find_workzone_tt_data(df_assets_recom, df_valid).reset_index(drop=True)
df_valid = df_tt_workzone.copy()
df_valid

16.03 %
32.05 %
48.08 %
64.1 %
80.13 %
96.15 %


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:25:14 PM,1.73,1.7,0.03,OK,2.750
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:30:15 PM,1.72,1.7,0.02,OK,2.750
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:35:20 PM,1.72,1.7,0.02,OK,2.750
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:40:18 PM,1.70,1.7,0.00,OK,2.750
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:45:15 PM,1.70,1.7,0.00,OK,2.750
...,...,...,...,...,...,...,...,...
2914135,86449733135,Reactivation,4/3/2022 6:44:37 AM,2.63,2.6,0.03,OK,3.693
2914136,86449733135,Reactivation,4/3/2022 6:42:34 AM,2.63,2.6,0.03,OK,3.693
2914137,86449733135,Reactivation,4/3/2022 6:40:37 AM,2.63,2.6,0.03,OK,3.693
2914138,86449733135,Reactivation,4/3/2022 6:38:37 AM,2.63,2.6,0.03,OK,3.693


In [18]:
# find_data_workzone(df_assets_all_active, file_name='TravelTimesPerWorkZone_AllActive.xlsx')
# find_data_workzone(df_assets_recom, file_name=''TravelTimesPerWorkZone_Recommended.xlsx')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_AllActive.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_all_active['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_all_active)*100, 2)} %')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_Recommended.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_recom['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_recom)*100, 2)} %')


## Extract New Columns

### Recalculate Delay Time & State

In [19]:
# Recalculate DelayTime 
df_valid['CalculatedDelayTime'] = (df_valid.UnderConstructionTime - df_valid.BaselineTime).clip(lower=0).round(4)
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:25:14 PM,1.73,1.7,0.03,OK,2.750,0.03
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:30:15 PM,1.72,1.7,0.02,OK,2.750,0.02
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:35:20 PM,1.72,1.7,0.02,OK,2.750,0.02
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:40:18 PM,1.70,1.7,0.00,OK,2.750,0.00
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,5/30/2021 8:45:15 PM,1.70,1.7,0.00,OK,2.750,0.00
...,...,...,...,...,...,...,...,...,...
2914135,86449733135,Reactivation,4/3/2022 6:44:37 AM,2.63,2.6,0.03,OK,3.693,0.03
2914136,86449733135,Reactivation,4/3/2022 6:42:34 AM,2.63,2.6,0.03,OK,3.693,0.03
2914137,86449733135,Reactivation,4/3/2022 6:40:37 AM,2.63,2.6,0.03,OK,3.693,0.03
2914138,86449733135,Reactivation,4/3/2022 6:38:37 AM,2.63,2.6,0.03,OK,3.693,0.03


In [20]:
# # Recalculate State
# df_valid['CalculatedState'] = np.where(df_valid['CalculatedDelayTime'] < DELAY_THRESH, 'OK', 'DELAY')
# df_valid

### From 'CreatedDate'

In [21]:
df_valid['CreatedDate'] = pd.to_datetime(df_valid['CreatedDate'])


In [22]:
# DayType:
#     1: Monday (0) & Friday (4)
#     2: OtherWeekday (1,2,3)
#     3: Weekend (5,6)
dict_day_type = {0:1, 1:2, 2:2, 3:2, 4:1, 5:3, 6:3}

# HourType: 
#     Peak (1): 6-9:59, 15-17:59
#     Off-Peak (2): 10-14:59, 18-5:59

dict_hour_type = {6:1, 7:1, 8:1,9:1,15:1,16:1,17:1,
                  10:2,11:2,12:2,13:2,14:2, 18:2,19:2, 20:2, 21:2, 22:2, 23:2, 0:2, 1:2, 2:2, 3:2, 4:2, 5:2}
# #     1: 6-10
# #     2: 10-15
# #     3: 15-18
# #     4: 18-22
# #     5: 22-6
# dict_hour_type = {6:1, 7:1, 8:1,9:1, 
#                   10:2,11:2,12:2,13:2,14:2,
#                   15:3,16:3,17:3,
#                   18:4,19:4,20:4,21:4,
#                   22:5,23:5,0:5,1:5,2:5,3:5,4:5,5:5}


In [23]:
df_valid['CreatedDate:Date'] = df_valid['CreatedDate'].dt.date
print('*** Date *** !DONE!')

# Return the day of the week as an integer, where Monday is 1 and Sunday is 8.
df_valid['CreatedDate:Weekday'] = df_valid['CreatedDate'].dt.weekday + 1
# df_valid['CreatedDate:DayName'] = df_valid['CreatedDate'].dt.day_name()
print('*** Weekday *** !DONE!')

# df_valid['CreatedDate:WeekOfYear'] = df_valid['CreatedDate'].dt.week_of_year
# df_valid['CreatedDate:DayOfYear'] = df_valid['CreatedDate'].dt.day_of_year

df_valid['CreatedDate:Year'] = df_valid['CreatedDate'].dt.year
print('*** Year *** !DONE!')
df_valid['CreatedDate:Month'] = df_valid['CreatedDate'].dt.month
print('*** Month *** !DONE!')
# # Between 1 and the number of days in the given month of the given year.
df_valid['CreatedDate:Day'] = df_valid['CreatedDate'].dt.day
print('*** Day *** !DONE!')
df_valid['CreatedDate:Hour'] = df_valid['CreatedDate'].dt.hour
print('*** Hour *** !DONE!')
# df_valid['CreatedDate:Minute'] = df_valid['CreatedDate'].dt.minute

# Seasons:
# 1: Winter – December, January and February.
# 2: Spring – March, April and May.
# 3: Summer – June, July and August.
# 4: Autumn – September, October and November.
df_valid['CreatedDate:Season'] = df_valid['CreatedDate'].dt.month%12 // 3 + 1
print('*** Season *** !DONE!')

df_valid['CreatedDate:DayType'] = df_valid['CreatedDate'].dt.weekday.map(dict_day_type)
print('*** DayType *** !DONE!')

df_valid['CreatedDate:HourType'] = df_valid['CreatedDate'].dt.hour.map(dict_hour_type)
print('*** HourType *** !DONE!')

df_valid

*** Date *** !DONE!
*** Weekday *** !DONE!
*** Year *** !DONE!
*** Month *** !DONE!
*** Day *** !DONE!
*** Hour *** !DONE!
*** Season *** !DONE!
*** DayType *** !DONE!
*** HourType *** !DONE!


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.7,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.7,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.7,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,86449733135,Reactivation,2022-04-03 06:44:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1
2914136,86449733135,Reactivation,2022-04-03 06:42:34,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1
2914137,86449733135,Reactivation,2022-04-03 06:40:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1
2914138,86449733135,Reactivation,2022-04-03 06:38:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1


In [24]:
# df_valid.to_csv(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.csv'), header = True, index=True)
# df_valid.to_excel(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.xlsx'), header = True, index=True)

### From DelayTime

In [25]:
print("MAX(DelayTime) =", df_valid['DelayTime'].max())
df_valid.describe().apply(lambda s: s.apply('{0:.5f}'.format))


MAX(DelayTime) = 106.16


,AssetNumber,UnderConstructionTime,BaselineTime,DelayTime,SegmentLength,CalculatedDelayTime,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType
count,2914140.00000,2914140.00000,2914100.00000,2914071.00000,2914025.00000,2914100.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000
mean,54435659328.01405,2.05770,1.97481,0.14096,3.28591,0.14093,3.56630,2021.53065,4.26783,15.84697,11.30900,2.13253,1.87625,1.67082
std,28637945985.64198,1.42825,1.28361,0.38792,4.38318,0.38790,1.86243,0.49906,1.03973,8.94153,6.42498,0.34275,0.69917,0.46992
min,10000204812.00000,0.78000,1.00000,0.00000,0.00000,0.00000,1.00000,2021.00000,2.00000,1.00000,0.00000,1.00000,1.00000,1.00000
25%,25290248946.00000,1.63000,1.63000,0.00000,2.70000,0.00000,2.00000,2021.00000,3.00000,9.00000,6.00000,2.00000,1.00000,1.00000
50%,53956931309.00000,1.87000,1.80000,0.00000,2.80000,0.00000,3.00000,2022.00000,4.00000,15.00000,11.00000,2.00000,2.00000,2.00000
75%,83015587391.00000,2.32000,2.17000,0.13000,3.41200,0.13000,5.00000,2022.00000,5.00000,24.00000,16.00000,2.00000,2.00000,2.00000
max,99994344960.00000,212.67000,189.90000,106.16000,270.69600,106.16000,7.00000,2022.00000,6.00000,31.00000,23.00000,3.00000,3.00000,2.00000


In [26]:
LEVEL_COUNT = 4

DELAY_THRESH = df_valid['DelayTime'].max() / 2 

DELAY_MIN_THRESH = 1
DELAY_MAX_THRESH = 5

In [27]:
print(f"LEVEL_COUNT = {LEVEL_COUNT}\n")
print("Class 1: DelayTime =", 0)

if LEVEL_COUNT == 3:
    df_valid['DelayLevel'] = (
        np.where(df_valid['DelayTime'] == 0.0, 1,
                 np.where(df_valid['DelayTime'] > DELAY_THRESH, 3, 2)))
    
    print("Class 2: DelayTime <=", DELAY_THRESH)
    print("Class 3: DelayTime >", DELAY_THRESH)
    print("\DELAY_THRESH =", DELAY_THRESH)

elif LEVEL_COUNT == 4:
    df_valid['DelayLevel'] = (
        np.where(df_valid['DelayTime'] == 0.0, 1,
                 np.where(df_valid['DelayTime'] > DELAY_MAX_THRESH, 4,
                          np.where(df_valid['DelayTime'] > DELAY_MIN_THRESH, 3, 2))))
    
    print(f"Class 2: 0 < DelayTime <= {DELAY_MIN_THRESH}")
    print(f"Class 3: {DELAY_MIN_THRESH} < DelayTime <= {DELAY_MAX_THRESH}")
    print("Class 4: DelayTime >", DELAY_MAX_THRESH)

    print("\nDELAY_MIN_THRESH =", DELAY_MIN_THRESH)
    print("DELAY_MAX_THRESH =", DELAY_MAX_THRESH)


# if cond1:
#     exp1
# elif cond2:
#     exp2
# else:
#     exp3
# np.where(cond1, exp1, np.where(cond2, exp2, ...))
df_valid

LEVEL_COUNT = 4

Class 1: DelayTime = 0
Class 2: 0 < DelayTime <= 1
Class 3: 1 < DelayTime <= 5
Class 4: DelayTime > 5

DELAY_MIN_THRESH = 1
DELAY_MAX_THRESH = 5


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.7,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.7,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.7,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,86449733135,Reactivation,2022-04-03 06:44:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2
2914136,86449733135,Reactivation,2022-04-03 06:42:34,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2
2914137,86449733135,Reactivation,2022-04-03 06:40:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2
2914138,86449733135,Reactivation,2022-04-03 06:38:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2


In [28]:
df_valid.loc[df_valid['DelayLevel'] == 2]    

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.7,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2
132,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-06-01 19:45:12,1.75,1.7,0.05,OK,2.750,0.05,2021-06-01,2,2021,6,1,19,3,2,2,2
133,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-06-01 19:50:13,1.77,1.7,0.07,OK,2.750,0.07,2021-06-01,2,2021,6,1,19,3,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,86449733135,Reactivation,2022-04-03 06:44:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2
2914136,86449733135,Reactivation,2022-04-03 06:42:34,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2
2914137,86449733135,Reactivation,2022-04-03 06:40:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2
2914138,86449733135,Reactivation,2022-04-03 06:38:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2


### Copy to df_tt_workzone

In [29]:
df_tt_workzone = df_valid.copy()
df_tt_workzone.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,AssetNumber,UnderConstructionTime,BaselineTime,DelayTime,SegmentLength,CalculatedDelayTime,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
count,2914140.00000,2914140.00000,2914100.00000,2914071.00000,2914025.00000,2914100.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000,2914140.00000
mean,54435659328.01405,2.05770,1.97481,0.14096,3.28591,0.14093,3.56630,2021.53065,4.26783,15.84697,11.30900,2.13253,1.87625,1.67082,1.51168
std,28637945985.64198,1.42825,1.28361,0.38792,4.38318,0.38790,1.86243,0.49906,1.03973,8.94153,6.42498,0.34275,0.69917,0.46992,0.54646
min,10000204812.00000,0.78000,1.00000,0.00000,0.00000,0.00000,1.00000,2021.00000,2.00000,1.00000,0.00000,1.00000,1.00000,1.00000,1.00000
25%,25290248946.00000,1.63000,1.63000,0.00000,2.70000,0.00000,2.00000,2021.00000,3.00000,9.00000,6.00000,2.00000,1.00000,1.00000,1.00000
50%,53956931309.00000,1.87000,1.80000,0.00000,2.80000,0.00000,3.00000,2022.00000,4.00000,15.00000,11.00000,2.00000,2.00000,2.00000,1.00000
75%,83015587391.00000,2.32000,2.17000,0.13000,3.41200,0.13000,5.00000,2022.00000,5.00000,24.00000,16.00000,2.00000,2.00000,2.00000,2.00000
max,99994344960.00000,212.67000,189.90000,106.16000,270.69600,106.16000,7.00000,2022.00000,6.00000,31.00000,23.00000,3.00000,3.00000,2.00000,4.00000


In [30]:
gc.collect()

479

# Get Weather Data

In [31]:
df_matched_weather_ids = pd.read_csv(os.path.join(ats_path, 'AssetNumberStationID.csv'), index_col=0)
df_matched_weather_ids.rename(columns = {'Asset Number': 'AssetNumber', 'Station ID': 'Weather:StationID'}, inplace=True)
df_matched_weather_ids

,AssetNumber,Weather:StationID
0,23023696729,706
1,98128214373,706
2,73174024548,706
3,40105790845,706
4,12738620179,706
...,...,...
619,55858030442,794
620,45364364608,794
621,40786939101,794
622,34684428742,49408


In [32]:
df_weather = pd.DataFrame()

# for station_id in df_matched_weather_ids['Weather:StationID'].unique():
for folder_name in os.listdir(weather_path):
    station_id = int(folder_name)
    weather_joined_files = glob.glob(os.path.join(weather_path, f'{station_id}/*.csv'))
    df_weather_station = load_csv_to_df(weather_joined_files)
    
    df_weather_station.insert(loc=0, column='StationID', value=station_id)
    df_weather_station['Date/Time (LST)'] = pd.to_datetime(df_weather_station['Date/Time (LST)'])
    # df_weather_station['Date'] = df_weather_station['Date/Time (LST)'].dt.date
    df_weather_station.insert(loc=df_weather_station.columns.get_loc('Date/Time (LST)')+1, column='Date', value= df_weather_station['Date/Time (LST)'].dt.date)
    df_weather_station.insert(loc=df_weather_station.columns.get_loc('Time (LST)')+1, column='Hour', value= df_weather_station['Date/Time (LST)'].dt.hour)

    df_weather = pd.concat([df_weather, df_weather_station[['StationID', 'Longitude (x)', 'Latitude (y)', 'Date', 'Hour', 'Temp (°C)', 'Precip. Amount (mm)', 'Wind Spd (km/h)']]], axis=0)
    
df_weather.rename(columns = {'Temp (°C)':'Temp',
                             'Precip. Amount (mm)':'Precip',
                             'Wind Spd (km/h)':'WindSpd',
                             'Longitude (x)':'Longitude',
                             'Latitude (y)':'Latitude'},
                  inplace=True)
df_weather.columns = ['Weather:' + str(col) for col in df_weather.columns]

df_weather

,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,707,-121.76,49.24,2022-07-01,0,NaN,NaN,NaN
1,707,-121.76,49.24,2022-07-01,1,NaN,NaN,NaN
2,707,-121.76,49.24,2022-07-01,2,NaN,NaN,NaN
3,707,-121.76,49.24,2022-07-01,3,NaN,NaN,NaN
4,707,-121.76,49.24,2022-07-01,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26299,8214,-119.29,52.13,2020-01-31,19,0.1,1.0,3.0
26300,8214,-119.29,52.13,2020-01-31,20,0.1,0.0,2.0
26301,8214,-119.29,52.13,2020-01-31,21,0.1,0.3,3.0
26302,8214,-119.29,52.13,2020-01-31,22,0.3,0.0,3.0


In [33]:
df_weather.nunique()

Weather:StationID      40
Weather:Longitude      30
Weather:Latitude       26
Weather:Date         1096
Weather:Hour           24
Weather:Temp          842
Weather:Precip        119
Weather:WindSpd        66
dtype: int64

## Add Weather Data to Traffic Time Data

In [34]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_weather_ids, left_on='AssetNumber', right_on='AssetNumber')
df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.7,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2,1355
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.7,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.7,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.7,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,86449733135,Reactivation,2022-04-03 06:44:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2,1355
2914136,86449733135,Reactivation,2022-04-03 06:42:34,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2,1355
2914137,86449733135,Reactivation,2022-04-03 06:40:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2,1355
2914138,86449733135,Reactivation,2022-04-03 06:38:37,2.63,2.6,0.03,OK,3.693,0.03,2022-04-03,7,2022,4,3,6,2,3,1,2,1355


In [35]:
df_tt_workzone = pd.merge(left = df_tt_workzone, right = df_weather,
                          left_on = ['Weather:StationID', 'CreatedDate:Date', 'CreatedDate:Hour'],
                          right_on = ['Weather:StationID', 'Weather:Date', 'Weather:Hour'])
df_tt_workzone = df_tt_workzone.drop(columns=['Weather:Date', 'Weather:Hour'])

df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,...,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,2021-05-30,...,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,...,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,...,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,...,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,...,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:35:11,1.92,1.93,0.00,OK,3.24,0.00,2021-05-26,...,2,2,2,1,10223,-118.03,52.93,17.5,0.0,14.0
2914136,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:40:16,1.92,1.93,0.00,OK,3.24,0.00,2021-05-26,...,2,2,2,1,10223,-118.03,52.93,17.5,0.0,14.0
2914137,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:45:12,1.92,1.93,0.00,OK,3.24,0.00,2021-05-26,...,2,2,2,1,10223,-118.03,52.93,17.5,0.0,14.0
2914138,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:50:13,1.92,1.93,0.00,OK,3.24,0.00,2021-05-26,...,2,2,2,1,10223,-118.03,52.93,17.5,0.0,14.0


### Checking Weather data

In [36]:
 df_weather[(df_weather['Weather:Date'] == df_tt_workzone.iloc[0]['CreatedDate:Date']) &
            (df_weather['Weather:Hour'] == 20) & 
            (df_weather['Weather:StationID'] == 1355)]
    
# df_tt_workzone.iloc[0]['CreatedDate:Date']

,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
23348,1355,-119.47,52.72,2021-05-30,20,NaN,NaN,NaN


In [37]:
df_matched_weather_ids[df_matched_weather_ids['Weather:StationID'] == 1355] 

,AssetNumber,Weather:StationID
84,71808680042,1355
85,51881464571,1355
86,53330524217,1355
87,61325418624,1355
88,50744787359,1355
...,...,...
167,30783661221,1355
168,92281098181,1355
169,10897197432,1355
170,64277836054,1355


In [38]:
df_weather[df_weather['Weather:Temp'].isnull()]

,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,707,-121.76,49.24,2022-07-01,0,NaN,NaN,NaN
1,707,-121.76,49.24,2022-07-01,1,NaN,NaN,NaN
2,707,-121.76,49.24,2022-07-01,2,NaN,NaN,NaN
3,707,-121.76,49.24,2022-07-01,3,NaN,NaN,NaN
4,707,-121.76,49.24,2022-07-01,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
23421,8214,-119.29,52.13,2022-10-31,21,NaN,NaN,NaN
23422,8214,-119.29,52.13,2022-10-31,22,NaN,NaN,NaN
23423,8214,-119.29,52.13,2022-10-31,23,NaN,NaN,NaN
26247,8214,-119.29,52.13,2020-01-29,15,NaN,NaN,NaN


In [39]:
# def find_weather_info(tt_entry, dict_tt_help_idx):
#     station_id = dict_matched_ids[tt_entry[dict_tt_help_idx['AssetNumber']]] 
#     weather_data = dict_weather[station_id]

#     weather_record = weather_data[(weather_data['Date/Time (LST)'].dt.date == tt_entry[dict_tt_help_idx['CreatedDate']].date())
#                                   & (weather_data['Date/Time (LST)'].dt.hour == tt_entry[dict_tt_help_idx['CreatedDate']].hour)]

#     return weather_record


# def dict_funct():
#     temp = df_tt_workzone.to_dict('records')
#     tt_idx = 0

#     for tt_entry in temp:
#         if tt_entry['AssetNumber'] in dict_matched_ids:
#             weather_record = find_weather_info(tt_entry)

#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Temp']] = weather_record['Temp (°C)'].values[0]
#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Precip']] = weather_record['Precip. Amount (mm)'].values[0]
#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:WindSpd']] = weather_record['Wind Spd (km/h)'].values[0]

#         if tt_idx % 10000 == 0:
#             print(f'{(tt_idx+1)/len(df_tt_workzone)*100:.2f}%')
#         tt_idx += 1

#     return df_tt_workzone


# def add_weather_data(df_tt, start, stop):
#     print(start, stop)
#     df_tt = df_tt.reset_index()

#     df_tt['Weather:Temp'] = ''
#     df_tt['Weather:Precip'] = ''
#     df_tt['Weather:WindSpd'] = ''

#     dict_tt_help_idx = {'Weather:Temp': df_tt.columns.get_loc('Weather:Temp'),
#                         'Weather:Precip': df_tt.columns.get_loc('Weather:Precip'),
#                         'Weather:WindSpd': df_tt.columns.get_loc('Weather:WindSpd'),
#                         'CreatedDate': df_tt.columns.get_loc('CreatedDate'),
#                         'AssetNumber': df_tt.columns.get_loc('AssetNumber')}
    
#     df_tt_workzone = dict_funct(df_tt, dict_tt_help_idx, start, stop)
#     df_tt_workzone.to_csv(os.path.join(output_root_path, f'processed_travel_time_{start}-{stop}.csv'))  
#     return df_tt_workzone

In [40]:
gc.collect()

186

# Get Counters Data


In [41]:
df_matched_counters_ids = pd.read_excel(os.path.join(counter_path, 'NearestCountStationsToAssets.xlsx'))[['Asset_Numb', 'prefixSiteNo']]
df_matched_counters_ids.rename(columns = {'Asset_Numb': 'AssetNumber', 'prefixSiteNo': 'Traffic:SiteNo'}, inplace=True)
df_matched_counters_ids

,AssetNumber,Traffic:SiteNo
0,23023696729,17-044EW
1,98128214373,17-044EW
2,25033704574,23-009NS
3,71808680042,23-010NS
4,27743462715,23-010NS
...,...,...
619,12066281555,21-011NS
620,62134790681,23-010NS
621,33379829945,23-010NS
622,21272146931,P-27-1EW


In [42]:
def clean_traffic_data(df):
    if df.shape[1] == 35:
        df = df.drop(columns=['Unnamed: 7', 'Unnamed: 31', 'Unnamed: 34'])
    elif df.shape[1] == 34: 
        df = df.drop(columns=[ 'Unnamed: 30', 'Unnamed: 33'])
    elif df.shape[1] == 32:
        df = df.drop(columns=['Unnamed: 28', 'Unnamed: 31'])
    elif df.shape[1] == 30:
        df = df.drop(columns=['Unnamed: 26', 'Unnamed: 29'])
    
    df = df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 9', 'Unnamed: 17'])
    df = df.dropna(how='all', axis=1)
    df = df.dropna(axis=0)
    
    df.rename(columns = {'Unnamed: 6':'Date'}, inplace = True)
    df = df.reset_index(drop=True)
    return df

In [43]:
ignore_stations = ['21-012NS', '23-005EW', 'W-23-2EW']

In [44]:
df_traffic = {'Temp': pd.DataFrame(), 'Perm': pd.DataFrame()}

# for site_no in df_matched_counters_ids['prefixSiteNo'].unique():
for site_no in os.listdir(os.path.join(counter_path, f'TrafficInfo')):
    if site_no in ignore_stations:
        continue
        
    site_type = 'DV01' if site_no.startswith('P') else 'DV03S' 
    
    counter_joined_files = glob.glob(os.path.join(counter_path, f'TrafficInfo/{site_no}/{site_type}*.xls'))
    for f in counter_joined_files:
        raw_traffic_site = pd.read_excel(f, header=5, thousands=',')
    
    df_traffic_site = pd.DataFrame()
#     df_traffic_site['Year'] = int(counter_joined_files[0][-8:-4])
# #     df_traffic_site['CreatedDate'].year - 1

    if site_no.startswith('P'):
#         print("\n****BINGO***")
#         print(site_no, raw_traffic_site.shape, '\n')
        clean_traffic_site = clean_traffic_data(raw_traffic_site)
        temp_datetime = pd.to_datetime(clean_traffic_site['Date'])
        df_traffic_site['BaseYear'] = temp_datetime.dt.year
        df_traffic_site['Month'] = temp_datetime.dt.month
        df_traffic_site['Day'] = temp_datetime.dt.day
        df_traffic_site['AADT'] = clean_traffic_site['Road'].astype(float)
        df_traffic_site.insert(loc=0, column='SiteNo', value=site_no)
        df_traffic_site.insert(loc=1, column='IsTemporary', value=False)
        df_traffic_site.insert(loc=len(df_traffic_site.columns), column='CreatedDate', value=pd.to_datetime(raw_traffic_site.iloc[-1][3]))
        df_traffic['Perm'] = pd.concat([df_traffic['Perm'], df_traffic_site], axis=0)
    else:
        temp_datetime = pd.to_datetime(counter_joined_files[0][-8:-4]).date()   # YEAR in datetime format    
        df_traffic_site['BaseYear'] = [temp_datetime.year]
        df_traffic_site['Month'] = [temp_datetime.month]
        df_traffic_site['Day'] = [temp_datetime.day]
        df_traffic_site['AADT'] = [float(raw_traffic_site.iloc[-1][13])]
        df_traffic_site.insert(loc=0, column='SiteNo', value=site_no)
        df_traffic_site.insert(loc=1, column='IsTemporary', value=True)
        df_traffic_site['CreatedDate'] = [pd.to_datetime(raw_traffic_site.iloc[-1][1]).date()]
        df_traffic['Temp'] = pd.concat([df_traffic['Temp'], df_traffic_site], axis=0)
        
for site_type in df_traffic:
    df_traffic[site_type].columns = ['Traffic:' + str(col) for col in df_traffic[site_type].columns]
    df_traffic[site_type] = df_traffic[site_type].reset_index(drop=True)

df_traffic['Perm']

,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
0,P-17-1EW,False,2021,1,1,9251.0,2022-03-08
1,P-17-1EW,False,2021,1,2,12042.0,2022-03-08
2,P-17-1EW,False,2021,1,3,13890.0,2022-03-08
3,P-17-1EW,False,2021,1,4,10507.0,2022-03-08
4,P-17-1EW,False,2021,1,5,10812.0,2022-03-08
...,...,...,...,...,...,...,...
1825,P-23-3NS,False,2021,12,27,1668.0,2022-03-09
1826,P-23-3NS,False,2021,12,28,1751.0,2022-03-09
1827,P-23-3NS,False,2021,12,29,1748.0,2022-03-09
1828,P-23-3NS,False,2021,12,30,1751.0,2022-03-09


In [45]:
df_traffic['Temp']

,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
0,27-001E,True,2013,1,1,9487.004,2014-01-20
1,21-937NS,True,2006,1,1,3767.000,2007-02-14
2,27-022E,True,2013,1,1,7449.458,2014-01-20
3,27-0404,True,2013,1,1,3232.165,2014-01-20
4,27-001W,True,2013,1,1,9435.631,2014-01-20
5,21-011NS,True,2017,1,1,1352.857,2018-01-25
6,21-013EW,True,2017,1,1,402.704,2018-01-25
7,23-001NS,True,2017,1,1,1542.281,2018-01-25
8,27-003EW,True,2013,1,1,3608.106,2014-01-20
9,21-006NS,True,2016,1,1,3140.103,2017-02-09


## Add Counters Data to Traffic Time Data

In [46]:
pd.set_option('display.max_columns', None)

In [47]:
GROWTH_RATE = 0.01

In [48]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_counters_ids, on='AssetNumber')
# df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_counters_ids, left_on='AssetNumber', right_on='AssetNumber')

df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,27040813369,Reactivation,2022-06-06 05:50:41,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS
2914136,27040813369,Reactivation,2022-06-06 05:48:40,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS
2914137,27040813369,Reactivation,2022-06-06 05:47:29,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS
2914138,27040813369,Reactivation,2022-06-06 05:44:41,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS


In [49]:
def find_diff(a, b):
    return list(set(a).difference(b))
    

# set(df_tt_workzone['Traffic:SiteNo'].unique()).difference(set(os.listdir(os.path.join(counter_path, f'TrafficInfo'))))
missing_site = find_diff(df_tt_workzone['Traffic:SiteNo'].unique(), os.listdir(os.path.join(counter_path, f'TrafficInfo')))

missing_data_count = len(df_tt_workzone[df_tt_workzone['Traffic:SiteNo'].isin(ignore_stations + missing_site)])
missing_data_count

157460

In [50]:
# tt = pd.merge(left = df_tt_workzone, right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='left')
# tt = pd.merge(left = df_tt_workzone, right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='inner')
join_temp_traffic = df_tt_workzone.reset_index().merge(
    df_traffic['Temp'], on = 'Traffic:SiteNo', how="inner").set_index('index')
join_temp_traffic

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2612925,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,2,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,1,1,10918.750,2014-01-20
2612926,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,2,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,1,1,10918.750,2014-01-20
2612927,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,2,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,1,1,10918.750,2014-01-20


In [51]:
# tp = pd.merge(left = df_tt_workzone, right = df_traffic['Perm'], 
#          left_on = ['Traffic:SiteNo', 'CreatedDate:Month', 'CreatedDate:Day'],
#          right_on = ['Traffic:SiteNo', 'Traffic:Month', 'Traffic:Day'],
#              how='inner')
join_perm_traffic = df_tt_workzone.reset_index().merge(df_traffic['Perm'], how="inner",
                                                       left_on = ['Traffic:SiteNo', 'CreatedDate:Month', 'CreatedDate:Day'],
                                                       right_on = ['Traffic:SiteNo', 'Traffic:Month', 'Traffic:Day']
                                                      ).set_index('index')
join_perm_traffic

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96679,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:25:12,3.10,3.00,0.10,OK,5.230,0.10,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
96680,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:30:13,3.10,3.00,0.10,OK,5.230,0.10,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
96681,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:35:18,3.07,3.00,0.07,OK,5.230,0.07,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
96682,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:40:16,3.07,3.00,0.07,OK,5.230,0.07,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
96683,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:45:14,3.07,3.00,0.07,OK,5.230,0.07,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1949958,21382867516,Spread 5A,2022-05-12 08:18:39,2.30,2.55,0.00,OK,2.538,0.00,2022-05-12,4,2022,5,12,8,2,2,1,1,1022,-120.80,50.11,NaN,NaN,NaN,P-27-1EW,False,2021,5,12,1973.0,2022-03-08
1949959,21382867516,Spread 5A,2022-05-12 08:16:37,2.25,2.55,0.00,OK,2.538,0.00,2022-05-12,4,2022,5,12,8,2,2,1,1,1022,-120.80,50.11,NaN,NaN,NaN,P-27-1EW,False,2021,5,12,1973.0,2022-03-08
1949960,21382867516,Spread 5A,2022-05-12 08:14:32,2.23,2.55,0.00,OK,2.538,0.00,2022-05-12,4,2022,5,12,8,2,2,1,1,1022,-120.80,50.11,NaN,NaN,NaN,P-27-1EW,False,2021,5,12,1973.0,2022-03-08


In [52]:
df_tt_workzone_join = pd.concat([join_perm_traffic, join_temp_traffic], axis=0)
df_tt_workzone_join['Traffic:AADT'] * (1 + (GROWTH_RATE * (df_tt_workzone_join['CreatedDate:Year'] - df_tt_workzone_join['Traffic:BaseYear'])))
df_tt_workzone_join = df_tt_workzone_join.drop(columns=['Traffic:Month', 'Traffic:Day', 'Traffic:CreatedDate'])
df_tt_workzone_join['Traffic:CalculatedAADT'] = df_tt_workzone_join['Traffic:AADT'] * (1 + (GROWTH_RATE * (df_tt_workzone_join['CreatedDate:Year'] - df_tt_workzone_join['Traffic:BaseYear'])))
df_tt_workzone_join

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:AADT,Traffic:CalculatedAADT
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96679,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:25:12,3.10,3.00,0.10,OK,5.23,0.10,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
96680,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:30:13,3.10,3.00,0.10,OK,5.23,0.10,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
96681,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:35:18,3.07,3.00,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
96682,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:40:16,3.07,3.00,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
96683,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:45:14,3.07,3.00,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2612925,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,2,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,10918.75,11792.25
2612926,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,2,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,10918.75,11792.25
2612927,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,2,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,10918.75,11792.25


In [53]:
incomplete_idx = find_diff(df_tt_workzone.index.values, df_tt_workzone_join.index.values)
incomplete_entries = df_tt_workzone[df_tt_workzone.index.isin(incomplete_idx)]
print('# incomplete_entries:', len(incomplete_entries))

df_tt_workzone = pd.concat([df_tt_workzone_join, incomplete_entries], axis=0).sort_index()
# df_tt_workzone = df_tt_workzone.drop(columns='AllTasksIndex')
df_tt_workzone

# incomplete_entries: 386761


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:AADT,Traffic:CalculatedAADT
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,27040813369,Reactivation,2022-06-06 05:50:41,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120
2914136,27040813369,Reactivation,2022-06-06 05:48:40,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120
2914137,27040813369,Reactivation,2022-06-06 05:47:29,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120
2914138,27040813369,Reactivation,2022-06-06 05:44:41,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120


In [54]:
df_tt_workzone.columns

Index(['AssetNumber', 'AssetIdentifier', 'CreatedDate',
       'UnderConstructionTime', 'BaselineTime', 'DelayTime', 'State',
       'SegmentLength', 'CalculatedDelayTime', 'CreatedDate:Date',
       'CreatedDate:Weekday', 'CreatedDate:Year', 'CreatedDate:Month',
       'CreatedDate:Day', 'CreatedDate:Hour', 'CreatedDate:Season',
       'CreatedDate:DayType', 'CreatedDate:HourType', 'DelayLevel',
       'Weather:StationID', 'Weather:Longitude', 'Weather:Latitude',
       'Weather:Temp', 'Weather:Precip', 'Weather:WindSpd', 'Traffic:SiteNo',
       'Traffic:IsTemporary', 'Traffic:BaseYear', 'Traffic:AADT',
       'Traffic:CalculatedAADT'],
      dtype='object')

In [55]:
# pd.merge(left = tp[:count], right = tt[:count], on = ['Traffic:SiteNo','Traffic:IsTemporary'], how='inner')
# pd.merge(left = tp[:count], right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='inner')

In [56]:
gc.collect()

63

# Get Speed Data

In [57]:
df_assets_speed = pd.read_csv(os.path.join(ats_path, 'AssetsNumberMaxSpeedALL.csv')).drop(columns=['Unnamed: 0'])
df_assets_speed.rename(columns = {'Asset_Numb': 'AssetNumber', 'maxspeed': 'MaxSpeed'}, inplace=True)
df_assets_speed


,AssetNumber,name,fclass,oneway,MaxSpeed
0,10000204812,Southern Yellowhead Highway,trunk,B,100.0
1,10201541400,Coquihalla Highway,motorway,F,120.0
2,10266236577,Southern Yellowhead Highway,trunk,B,100.0
3,11042076244,Yellowhead Highway,trunk,B,100.0
4,11181395533,Southern Yellowhead Highway,trunk,B,100.0
...,...,...,...,...,...
589,12285083600,NaN,NaN,NaN,NaN
590,55324137435,NaN,NaN,NaN,NaN
591,19316607964,NaN,NaN,NaN,NaN
592,61187226602,NaN,NaN,NaN,NaN


## Add Speed Data to Traffic Time Data

In [58]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_assets_speed[['AssetNumber', 'MaxSpeed']], on='AssetNumber')
df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:AADT,Traffic:CalculatedAADT,MaxSpeed
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176,NaN
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176,NaN
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,2,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176,NaN
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176,NaN
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,2,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2816070,27040813369,Reactivation,2022-06-06 05:50:41,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120,100.0
2816071,27040813369,Reactivation,2022-06-06 05:48:40,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120,100.0
2816072,27040813369,Reactivation,2022-06-06 05:47:29,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120,100.0
2816073,27040813369,Reactivation,2022-06-06 05:44:41,1.60,1.62,0.00,OK,2.792,0.00,2022-06-06,1,2022,6,6,5,3,1,2,1,10223,-118.03,52.93,7.0,0.0,5.0,23-010NS,True,2017.0,1900.944,1995.99120,100.0


In [59]:
gc.collect()

21

# Exploratory Data Analysis

## General Info about dataframe

In [60]:
df_tt_workzone.describe().to_csv(os.path.join(output_root_path, 'data_descriptive_statistics_PYTHON.csv'))
df_tt_workzone.describe().apply(lambda s: s.apply('{0:.5f}'.format))
# df_tt_workzone.describe(include='all')
# df_tt_workzone.describe(include='object')

,AssetNumber,UnderConstructionTime,BaselineTime,DelayTime,SegmentLength,CalculatedDelayTime,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:BaseYear,Traffic:AADT,Traffic:CalculatedAADT,MaxSpeed
count,2816075.00000,2816075.00000,2816040.00000,2816011.00000,2815966.00000,2816040.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,2816075.00000,1183962.00000,1053617.00000,1183962.00000,2437064.00000,2437064.00000,2437064.00000,2347776.00000
mean,54762552887.82471,2.06625,1.98409,0.14048,3.30773,0.14045,3.56477,2021.53244,4.26093,15.84821,11.30786,2.13059,1.87407,1.67128,1.51195,12498.93935,-119.89565,51.60269,8.84302,0.10343,7.67171,2015.50665,2120.62743,2249.59159,101.67119
std,28619895209.88954,1.44888,1.30381,0.38994,4.45686,0.38991,1.86437,0.49895,1.03837,8.90475,6.43114,0.34078,0.70008,0.46975,0.54621,20121.30842,1.10192,1.36695,6.33683,0.43486,5.43916,3.17074,2053.85696,2238.97018,7.34382
min,10000204812.00000,0.85000,1.00000,0.00000,0.00000,0.00000,1.00000,2021.00000,2.00000,1.00000,0.00000,1.00000,1.00000,1.00000,1.00000,706.00000,-121.76000,49.24000,-15.40000,0.00000,0.00000,2009.00000,402.70400,422.83920,70.00000
25%,26620220646.00000,1.62000,1.63000,0.00000,2.70100,0.00000,2.00000,2021.00000,3.00000,9.00000,6.00000,2.00000,1.00000,1.00000,1.00000,1237.00000,-120.18000,51.29000,5.60000,0.00000,3.00000,2016.00000,1542.28100,1603.97224,100.00000
50%,53956931309.00000,1.88000,1.80000,0.00000,2.84700,0.00000,3.00000,2022.00000,4.00000,15.00000,11.00000,2.00000,2.00000,2.00000,1.00000,1355.00000,-119.47000,52.13000,9.40000,0.00000,7.00000,2017.00000,1900.94400,1976.98176,100.00000
75%,84067785913.00000,2.32000,2.18000,0.13000,3.50900,0.13000,5.00000,2022.00000,5.00000,24.00000,16.00000,2.00000,2.00000,2.00000,2.00000,10223.00000,-119.29000,52.72000,12.80000,0.00000,11.00000,2017.00000,1900.94400,1995.99120,100.00000
max,99994344960.00000,212.67000,189.90000,106.16000,270.69600,106.16000,7.00000,2022.00000,6.00000,31.00000,23.00000,3.00000,3.00000,2.00000,4.00000,53423.00000,-118.03000,52.93000,29.90000,7.70000,41.00000,2021.00000,10918.75000,11901.43750,120.00000


In [61]:
df_tt_workzone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2816075 entries, 0 to 2816074
Data columns (total 31 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   AssetNumber             int64         
 1   AssetIdentifier         object        
 2   CreatedDate             datetime64[ns]
 3   UnderConstructionTime   float64       
 4   BaselineTime            float64       
 5   DelayTime               float64       
 6   State                   object        
 7   SegmentLength           float64       
 8   CalculatedDelayTime     float64       
 9   CreatedDate:Date        object        
 10  CreatedDate:Weekday     int64         
 11  CreatedDate:Year        int64         
 12  CreatedDate:Month       int64         
 13  CreatedDate:Day         int64         
 14  CreatedDate:Hour        int64         
 15  CreatedDate:Season      int64         
 16  CreatedDate:DayType     int64         
 17  CreatedDate:HourType    int64         
 18  De

In [62]:
df_tt_workzone.isnull().sum()

AssetNumber                     0
AssetIdentifier                 0
CreatedDate                     0
UnderConstructionTime           0
BaselineTime                   35
DelayTime                      64
State                         161
SegmentLength                 109
CalculatedDelayTime            35
CreatedDate:Date                0
CreatedDate:Weekday             0
CreatedDate:Year                0
CreatedDate:Month               0
CreatedDate:Day                 0
CreatedDate:Hour                0
CreatedDate:Season              0
CreatedDate:DayType             0
CreatedDate:HourType            0
DelayLevel                      0
Weather:StationID               0
Weather:Longitude               0
Weather:Latitude                0
Weather:Temp              1632113
Weather:Precip            1762458
Weather:WindSpd           1632113
Traffic:SiteNo                  0
Traffic:IsTemporary        379011
Traffic:BaseYear           379011
Traffic:AADT               379011
Traffic:Calcul

# Export 

In [63]:
temp = df_tt_workzone.copy()
temp.columns = temp.columns.str.replace("[#,@,&,:,-,.,',[,]", '_')
pyreadstat.write_sav(temp, 'TravelTime_Valid_Recom_Weather_Traffic.sav')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [64]:
df_tt_workzone.to_csv(os.path.join(output_root_path, 'TravelTime_Valid_Recom_Weather_Traffic.csv'))
# df_tt_workzone.to_csv(os.path.join(output_root_path, 'TravelTime_Valid_Recom_Weather_Traffic+.csv'), header = True, index=True)

In [65]:
def find_delay_diff():
    diff_delay = df_valid[df_valid['DelayTime'] != df_valid['CalculatedDelayTime']]
    diff_delay.to_csv(os.path.join(output_root_path, 'wrong_DelayTime.csv'), header = True, index=True)
    return diff_delay

In [66]:
def find_state_diff():
    diff_state = df_valid[df_valid['State'] != df_valid['CalculatedState']]
    diff_state.to_csv(os.path.join(output_root_path, 'wrong_State.csv'), header = True, index=True)
    return diff_state 

## Plot

In [67]:
import seaborn as sns
import matplotlib.pyplot as plt
# from PIL import Image

%matplotlib inline
sns.set_style("darkgrid")
plt.style.use('ggplot')

STEP_SIZE = 0.5


In [68]:
def draw_time_plot(loc, loc_id, time_type, plt_name):
    fig, ax = plt.subplots()

    # loc.plot.scatter(x='CreatedDate:DayName', y=['UnderConstructionTime', 'BaselineTime'])
    # loc.plot.scatter(x='CreatedDate:DayName', y='UnderConstructionTime')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['UnderConstructionTime'], color='purple', marker='o', label='UnderConstructionTime (After)')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['BaselineTime'], color='orange', marker='h', label='BaselineTime (Before)')

    x, y  = [], []
    
    if time_type == 'Hour':
        for hour in range(24):
            loc_hour = loc.loc[loc[f'CreatedDate:{time_type}'] == hour]
            x.append(hour)
            y.append(loc_hour['UnderConstructionTime'].mean())
    
    elif time_type == 'DayName':
         for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
            loc_day = loc.loc[loc[f'CreatedDate:{time_type}'] == day]
            x.append(day)
            y.append(loc_day['UnderConstructionTime'].mean())
    
    temp = loc['BaselineTime'].mean()
    plt.plot(x, y, color='purple', marker='o', label='UnderConstructionTime (After)')
    plt.plot(x, [temp for i in range(len(y))], color='orange', marker='h', label='BaselineTime (Before)')
    
#     start, end = ax.get_ylim()
#     plt.yticks(np.arange(np.round(start), np.round(end)+STEP_SIZE, step=STEP_SIZE))
    if time_type == 'Hour':
        plt.xticks(np.arange(0, 24, step=1))
    
    fig.set_size_inches(14, 10)
    ax.legend()
    ax.grid(True)
    plt.title(f'Travel Time for AssetID {loc_id} ({time_type})')
    plt.xlabel(time_type)
    plt.ylabel('TravelTime')
    plt.tight_layout()
    plt.savefig(f'{output_root_path}/{plt_name}.png')
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


### Draw plot for selected asset ids

In [69]:
locations_id = [80002045894, 50253718586, 44853071770, 12738620179, 37528363723, 43805438913, 68881926366, 36674989230, 34592545857, 22685608926]


In [70]:
# for location_id in locations_id:
#     for time_type in ['DayName', 'Hour']:
#         location = df_valid.loc[df_valid['AssetNumber'] == location_id]
#         draw_time_plot(location, location_id, time_type, plt_name=f'{location_id}_{time_type}')


In [71]:
!zip ouput.zip ../working/*.sav

  adding: ../working/TravelTime_Valid_Recom_Weather_Traffic.sav (deflated 98%)
